In [2]:
import matplotlib 

matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

import matplotlib.pyplot as plt

SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [3]:
#import necessary packages
import subprocess
import numpy as np
import os.path
import os
import sys
import glob as glob
import matplotlib.pyplot as plt

class cd:
    """Context manager for changing the current working directory"""
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)

In [4]:
Ns      = np.arange(2, 12, 1)

In [3]:
#run the program using o2 optimization flag

subprocess.run(["gfortran", "Ex9-Segalini-CODE.f90", "-llapack", "-o", "./run.exe", "-O2"])

# path = "./data/"

#vector containing execution times 
times    = np.zeros((len(Ns), 2))

for i in range(len(Ns)):
    # lambda is set equal to 0.5
    output = subprocess.run(['./run.exe'], stdout=subprocess.PIPE, 
                            input="{}\n{}\n".format(Ns[i], 0.5), encoding="ascii")
    lines = output.stdout.lstrip().split('\n')
    #print(lines)
    times[i, 0] = float(lines[-2]) #time for computing eigenvalues
    times[i, 1] = float(lines[-4]) #time for computing H

    print('N = ', Ns[i])
    print('T_evl = ', times[i, 0], "T_H = ", times[i, 1])


N =  2
T_evl =  6.199999999999999e-05 T_H =  2.3999999999999716e-05
N =  3
T_evl =  8.099999999999991e-05 T_H =  3.000000000000008e-05
N =  4
T_evl =  9.200000000000007e-05 T_H =  6.700000000000022e-05
N =  5
T_evl =  0.00013000000000000077 T_H =  0.0002519999999999996
N =  6
T_evl =  0.000353 T_H =  0.0010869999999999999
N =  7
T_evl =  0.001424 T_H =  0.007319
N =  8
T_evl =  0.0078119999999999995 T_H =  0.05553399999999999
N =  9
T_evl =  0.05172399999999999 T_H =  0.46691
N =  10
T_evl =  0.438755 T_H =  3.549842
N =  11
T_evl =  3.0675109999999997 T_H =  31.329677


In [5]:
# np.savetxt("./times.dat", times, fmt='%f')
times = np.loadtxt("times.dat")

In [6]:
fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(6.68, 2.20))
#fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(14, 6))

ax1.scatter(Ns, times[:, 0], c='r')
ax2.scatter(Ns, times[:, 1], c='r')

ax1.set_xlabel('Number of particles $N$')
ax2.set_xlabel('Number of particles $N$')
ax1.set_ylabel('Execution time [s]')

ax1.set_title('Eigenvalues computation')
ax2.set_title('$\mathcal{H}$ construction')

plt.tight_layout()
plt.show()
plt.savefig('times.pgf')

D:\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  from ipykernel import kernelapp as app


In [7]:
# select some values of N for energy analysis
Nn = np.arange(3, 10, 1)
lambdas = np.linspace(0, 3, num=301)

In [7]:
# move to data folder
datapath = './data/'

# run the program for all the lambda-N combinations required
# sort data in appropriate folders
with cd(datapath):
    for i in range(len(Nn)):
        path = str(Nn[i])
        subprocess.run(["mkdir", path])
        subprocess.run(["cp", "run.exe", path])
        with cd(path):
            print("N = ", Nn[i])
            for j in range(len(lambdas)):
                subprocess.run(['./run.exe'], stdout=subprocess.PIPE, input="{}\n{}\n".format(Nn[i], lambdas[j]),
                                encoding="ascii")


N =  3
N =  4
N =  5
N =  6
N =  7
N =  8
N =  9


In [8]:
# vector for storing the first 4 energy levels, for each N considered, as a function of lambda
en_levels = np.zeros((len(Nn), len(lambdas), 4))

# extract first 4 eigenvalues 
for i in range(len(Nn)):
    path = './data/'+str(Nn[i])
    with cd(path):
        filenames = sorted(glob.glob('*.dat'))
        for j, filename in enumerate(filenames):
            with open(filename) as f:
                #print("FILE = ", filename)   
                en_levels[i, j, 0] = float(f.readline())
                #print("e0=", en_levels[i, j, 0])
                en_levels[i, j, 1] = float(f.readline())
                #print("e1=", en_levels[i, j, 1])
                en_levels[i, j, 2] = float(f.readline())
                #print("e2=", en_levels[i, j, 2])
                en_levels[i, j, 3] = float(f.readline())
                #print("e3=", en_levels[i, j, 3])
                

In [9]:
fig, ax = plt.subplots(2, 2, figsize=(6.68, 4))
#fig, ax = plt.subplots(2, 2, figsize=(14, 10))

for n in range(len(Nn)):
    ax[0, 0].plot(lambdas, en_levels[n, :, 0]/(Nn[n]-1))
    ax[0, 1].plot(lambdas, en_levels[n, :, 1]/(Nn[n]-1))
    ax[1, 0].plot(lambdas, en_levels[n, :, 2]/(Nn[n]-1))
    ax[1, 1].plot(lambdas, en_levels[n, :, 3]/(Nn[n]-1))

ax[1, 0].set_xlabel("External interaction strength $\lambda$")
ax[1, 1].set_xlabel("External interaction strength $\lambda$")

ax[0, 0].set_ylabel("$E_0/(N-1)$")
ax[0, 1].set_ylabel("$E_1/(N-1)$")
ax[1, 0].set_ylabel("$E_2/(N-1)$")
ax[1, 1].set_ylabel("$E_3/(N-1)$")


ax[0, 0].legend(Nn, title="$N$", loc='lower left')
ax[1, 0].legend(Nn, title="$N$", loc='lower left')
ax[1, 1].legend(Nn, title="$N$", loc='lower left')
ax[0, 1].legend(Nn, title="$N$", loc='lower left')

ax2.set_title('N=3')

plt.tight_layout()
plt.show()
plt.savefig('4evl_lambda_N.pgf')

D:\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.


In [10]:
fig, ax = plt.subplots(2, 2, figsize=(6.68, 4))
#fig, ax = plt.subplots(2, 2, figsize=(14, 10))

for n in range(len(Nn)):
    ax[0, 0].plot(lambdas, en_levels[n, :, 0]/(Nn[n]-1))
    ax[0, 1].plot(lambdas, en_levels[n, :, 1]/(Nn[n]-1))
    ax[1, 0].plot(lambdas, en_levels[n, :, 2]/(Nn[n]-1))
    ax[1, 1].plot(lambdas, en_levels[n, :, 3]/(Nn[n]-1))

ax[1, 0].set_xlabel("External interaction strength $\lambda$")
ax[1, 1].set_xlabel("External interaction strength $\lambda$")

#ax[0,0].set_xscale('log')

ax[0, 0].set_ylabel("$E_0/(N-1)$")
ax[0, 1].set_ylabel("$E_1/(N-1)$")
ax[1, 0].set_ylabel("$E_2/(N-1)$")
ax[1, 1].set_ylabel("$E_3/(N-1)$")

ax[0, 0].legend(Nn, title="$N$", loc='lower left')
ax[1, 0].legend(Nn, title="$N$", loc='lower left')
ax[1, 1].legend(Nn, title="$N$", loc='lower left')
ax[0, 1].legend(Nn, title="$N$", loc='lower left')

ax2.set_title('N=3')

plt.tight_layout()
plt.show()
plt.savefig('4evl_lambda_N.pgf')

D:\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.


In [11]:
fig, ax = plt.subplots(1, 3, figsize=(6.68, 2.2))
#fig, ax = plt.subplots(1, 3, figsize=(14, 5))

for i in range(4):
    ax[0].plot(lambdas, en_levels[0, :, i]/2)
    ax[1].plot(lambdas, en_levels[3, :, i]/5)
    ax[2].plot(lambdas, en_levels[6, :, i]/8)
    

ax[0].set_ylabel("$E_k/(N-1)$")

for i in range(3):
    ax[i].set_xlabel("External interaction strength $\lambda$")
    ax[i].set_xscale('log')
    title = "$N="+str(3*(i+1))+"$"
    ax[i].legend(("$E_0$", "$E_1$", "$E_2$", "$E_3$"), title=title, loc='lower left')
    ax[i].vlines(1, -5, 0.5, 'k', 'dashed')
    ax[i].set_ylim(-4,0)
    
plt.tight_layout()
plt.show()
plt.savefig('3N_evl_lambda.pgf')

D:\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.


In [12]:
fig, ax = plt.subplots(1, 2, figsize=(6.68, 2.2))
#fig, ax = plt.subplots(1, 2, figsize=(12, 4))

for n in range(len(Nn)):
    ax[0].plot(lambdas, en_levels[n, :, 1]-en_levels[n, :, 0])
    ax[1].plot(lambdas, en_levels[n, :, 2]-en_levels[n, :, 0])

ax[0].set_ylabel("Energy gap")

ax[0].set_title("$E_1 - E_0$")
ax[1].set_title("$E_2 - E_0$")

for i in range(2):
    ax[i].legend(Nn, title="$N$", loc='upper left')
    ax[i].set_xscale('log')
    #ax[i].set_yscale('log')
    ax[i].set_xlabel("External interaction strength $\lambda$")
    

plt.tight_layout()
plt.show()
plt.savefig('energygap.pgf')

D:\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
